In [22]:
import pandas as pd
import numpy as np

In [10]:
import httpx
from selectolax.parser import HTMLParser
import re
import pandas as pd

base_url = 'https://www.vlr.gg'

event_url = '/event/matches/1188/champions-tour-2023-lock-in-s-o-paulo/?series_id=all'

res = httpx.get(base_url+event_url)

html = HTMLParser(res.text)

matches = html.css('a.wf-module-item')

completed_matches = [match for match in matches if match.css_first('div.ml-status').text() == 'Completed']

for match in completed_matches:
    match_res = httpx.get(match_url := f'{base_url}{match.attributes["href"]}')

    match_html = HTMLParser(match_res.text)

    match_name = match_html.css_first('title').text().strip().split(' | ')[0].split(' vs. ')

    messy_match_result = match_html.css_first('div.js-spoiler').text()
    match_result = ' ' + ''.join(re.findall("\S+",messy_match_result)) + ' '

    print(match_sum:=match_result.join(match_name))

    tables = match_html.css('table.wf-table-inset')

    # for table in tables:
    #     for t in pd.read_html(table.html):
    #         for key,df in get_clean_df(t).items():
    #             display(df)

KOI 0:2 NRG Esports
DetonatioN FocusMe 0:2 Giants Gaming
FunPlus Phoenix 1:2 Karmine Corp
BBL Esports 1:2 DRX
Cloud9 2:0 Paper Rex
Team Heretics 0:2 Evil Geniuses
MIBR 0:2 Talon Esports
Gen.G 0:2 LOUD
NRG Esports 2:1 Giants Gaming
LOUD 2:0 Karmine Corp
DRX 2:1 Cloud9
Evil Geniuses 0:2 Talon Esports
NRG Esports 1:2 LOUD
DRX 2:1 Talon Esports
Team Liquid 0:2 Team Secret
Natus Vincere 2:0 KRÜ Esports
ZETA DIVISION 0:2 Leviatán
Team Vitality 2:1 Global Esports
FUT Esports 2:0 Rex Regum Qeon
100 Thieves 2:1 EDward Gaming
Sentinels 0:2 FNATIC
T1 0:2 FURIA
Team Secret 0:2 Natus Vincere
Leviatán 2:0 Team Vitality
FUT Esports 1:2 100 Thieves
FNATIC 2:0 FURIA
Natus Vincere 2:0 Leviatán
100 Thieves 0:2 FNATIC
LOUD 3:2 DRX
Natus Vincere 0:3 FNATIC
Team tarik 0:1 Team FRTTT
LOUD 2:3 FNATIC


In [16]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  5 non-null      object 
 1   Unnamed: 1  0 non-null      float64
 2   R           5 non-null      object 
 3   ACS         5 non-null      object 
 4   K           5 non-null      object 
 5   D           5 non-null      object 
 6   A           5 non-null      object 
 7   +/–         5 non-null      object 
 8   KAST        5 non-null      object 
 9   ADR         5 non-null      object 
 10  HS%         5 non-null      object 
 11  FK          5 non-null      object 
 12  FD          5 non-null      object 
 13  +/–.1       5 non-null      object 
dtypes: float64(1), object(13)
memory usage: 692.0+ bytes


In [15]:
sample_df['D'].map(lambda r : r[1:-1].strip().split(' ')[0])

0    12
1    12
2    10
3    13
4    15
Name: D, dtype: object

In [22]:
sample_df['Unnamed: 0'].map(lambda n : n.split(' '))

0    [crashies, NRG]
1      [ardiis, NRG]
2         [FNS, NRG]
3         [s0m, NRG]
4      [Victor, NRG]
Name: Unnamed: 0, dtype: object

In [17]:
def clean_df(df,func):
    result_df = pd.DataFrame()
    
    result_df['Name'] = df['Unnamed: 0'].map(lambda n : n.split(' ')[0])
    result_df['Team'] = df['Unnamed: 0'].map(lambda n : n.split(' ')[1])
    result_df['R'] = df['R'].map(func)
    result_df['ACS'] = df['ACS'].map(func)
    result_df['K'] = df['K'].map(func)
    result_df['D'] = df['D'].map(lambda d : d[1:-1].strip()).map(func)
    result_df['A'] = df['A'].map(func)
    result_df['KD_Diff'] = df['+/–'].map(func).map(lambda d : int(d))
    result_df['KAST%'] = df['KAST'].map(func).map(lambda p : int(p[:-1])/100)
    result_df['ADR'] = df['ADR'].map(func)
    result_df['HS%'] = df['HS%'].map(func).map(lambda p : int(p[:-1])/100)
    result_df['FK'] = df['FK'].map(func)
    result_df['FD'] = df['FD'].map(func)
    result_df['FKFD_Diff'] = df['+/–.1'].map(func)
    
    
    
    return result_df

In [5]:
clean_df(sample_df,lambda s : s.split(' ')[0])

NameError: name 'sample_df' is not defined

In [29]:
def get_clean_df(df):
    result = {}
    def get_all(s):
        try:
            return s.split(' ')[0]
        except AttributeError:
            return '00'
    
    def get_atk(s):
        try:
            return s.split(' ')[2]
        except AttributeError:
            return '00'
    
    def get_def(s):
        try:
            return s.split(' ')[4]
        except AttributeError:
            return '00'
        
     
    
    
    
    for side in ['all','atk','def']:
        result[side] = clean_df(df,eval(f'get_{side}'))
        
    return result
        
    

In [25]:
for k,v in get_clean_df(sample_df).items():
    display(v)

NameError: name 'sample_df' is not defined

In [85]:
sample_df['+/–'].map(lambda x : x.split(' ')).info()

<class 'pandas.core.series.Series'>
RangeIndex: 5 entries, 0 to 4
Series name: +/–
Non-Null Count  Dtype 
--------------  ----- 
5 non-null      object
dtypes: object(1)
memory usage: 172.0+ bytes


In [20]:
sample_match = completed_matches[21]

In [30]:
match_res = httpx.get(match_url := f'{base_url}{sample_match.attributes["href"]}')

match_html = HTMLParser(match_res.text)

match_name = match_html.css_first('title').text().strip().split(' | ')[0].split(' vs. ')

messy_match_result = match_html.css_first('div.js-spoiler').text()
match_result = ' ' + ''.join(re.findall("\S+",messy_match_result)) + ' '
print(match_sum:=match_result.join(match_name))

tables = match_html.css('table.wf-table-inset')

for table in tables:
    for t in pd.read_html(table.html):
        for key,df in get_clean_df(t).items():
            display(df)

T1 0:2 FURIA


,Name,Team,R,ACS,K,D,A,KD_Diff,KAST%,ADR,HS%,FK,FD,FKFD_Diff
0,Sayaplayer,T1,0.83,196,14,17,0,-3,0.70,137,0.50,5,5,0
1,Munchkin,T1,0.78,157,11,15,9,-4,0.55,101,0.18,0,5,-5
2,Carpe,T1,0.77,172,12,17,4,-5,0.60,115,0.28,0,0,0
3,ban,T1,0.74,194,13,15,2,-2,0.75,118,0.18,2,0,+2
4,xeta,T1,0.71,172,10,16,14,-6,0.75,106,0.29,0,3,-3


,Name,Team,R,ACS,K,D,A,KD_Diff,KAST%,ADR,HS%,FK,FD,FKFD_Diff
0,Sayaplayer,T1,0.50,103,3,7,0,-4,0.63,78,0.67,1,2,-1
1,Munchkin,T1,0.19,95,2,8,2,-6,0.38,61,0.15,0,2,-2
2,Carpe,T1,0.28,94,3,8,2,-5,0.63,67,0.22,0,0,0
3,ban,T1,0.91,273,7,6,1,1,0.88,163,0.17,0,0,0
4,xeta,T1,0.70,229,5,7,6,-2,0.75,134,0.32,0,3,-3


,Name,Team,R,ACS,K,D,A,KD_Diff,KAST%,ADR,HS%,FK,FD,FKFD_Diff
0,Sayaplayer,T1,1.05,259,11,10,0,1,0.75,176,0.44,4,3,+1
1,Munchkin,T1,1.18,199,9,7,7,2,0.67,127,0.19,0,3,-3
2,Carpe,T1,1.10,224,9,9,2,0,0.58,148,0.30,0,0,0
3,ban,T1,0.63,143,6,9,1,-3,0.67,87,0.19,2,0,+2
4,xeta,T1,0.71,135,5,9,8,-4,0.75,88,0.27,0,0,0


,Name,Team,R,ACS,K,D,A,KD_Diff,KAST%,ADR,HS%,FK,FD,FKFD_Diff
0,Mazin,FUR,1.46,205,16,9,10,7,0.8,124,0.27,1,1,0
1,mwzera,FUR,1.38,268,18,13,11,5,0.9,183,0.12,4,0,+4
2,dgzin,FUR,1.32,291,21,13,3,8,0.7,181,0.31,4,4,0
3,Quick,FUR,1.07,199,14,14,3,0,0.8,141,0.34,2,2,0
4,Khalil,FUR,1.05,155,11,11,9,0,0.7,86,0.40,2,0,+2


...

,Name,Team,R,ACS,K,D,A,KD_Diff,KAST%,ADR,HS%,FK,FD,FKFD_Diff
0,Sayaplayer,T1,1.05,259,11,10,0,1,0.75,176,0.44,4,3,+1
1,Munchkin,T1,1.18,199,9,7,7,2,0.67,127,0.19,0,3,-3
2,Carpe,T1,1.10,224,9,9,2,0,0.58,148,0.30,0,0,0
3,ban,T1,0.63,143,6,9,1,-3,0.67,87,0.19,2,0,+2
4,xeta,T1,0.71,135,5,9,8,-4,0.75,88,0.27,0,0,0


,Name,Team,R,ACS,K,D,A,KD_Diff,KAST%,ADR,HS%,FK,FD,FKFD_Diff
0,Mazin,FUR,1.46,205,35,18,17,17,0.8,124,0.27,1,1,0
1,mwzera,FUR,1.38,268,32,24,20,8,0.9,183,0.12,4,0,+4
2,dgzin,FUR,1.32,291,37,20,6,17,0.7,181,0.31,4,4,0
3,Quick,FUR,1.07,199,35,24,4,11,0.8,141,0.34,2,2,0
4,Khalil,FUR,1.05,155,16,20,13,-4,0.7,86,0.40,2,0,+2


,Name,Team,R,ACS,K,D,A,KD_Diff,KAST%,ADR,HS%,FK,FD,FKFD_Diff
0,Mazin,FUR,1.41,231,11,6,4,5,0.75,147,0.36,0,1,-1
1,mwzera,FUR,1.24,233,10,7,3,3,0.83,156,0.17,1,0,+1
2,dgzin,FUR,0.81,257,10,11,1,-1,0.58,158,0.37,3,4,-1
3,Quick,FUR,1.01,185,8,8,0,0,0.75,139,0.33,1,1,0
4,Khalil,FUR,0.87,153,5,8,6,-3,0.67,94,0.40,1,0,+1


,Name,Team,R,ACS,K,D,A,KD_Diff,KAST%,ADR,HS%,FK,FD,FKFD_Diff
0,Mazin,FUR,1.52,168,5,3,6,2,0.88,91,0.13,1,0,+1
1,mwzera,FUR,1.60,322,8,6,8,2,1.00,223,0.08,3,0,+3
2,dgzin,FUR,2.09,343,11,2,2,9,0.88,217,0.25,1,0,+1
3,Quick,FUR,1.15,222,6,6,3,0,0.88,145,0.35,1,1,0
4,Khalil,FUR,1.32,159,6,3,3,3,0.75,74,0.40,1,0,+1


IndexError: list index out of range